In [1]:
# Cell 1 — setup & helper
import json, requests, pandas as pd
from pprint import pprint

AGENT = "http://127.0.0.1:8000/collect"

def call_agent(intent, args=None):
    args = args or {}
    r = requests.post(AGENT, json={"intent": intent, "args": args}, timeout=30)
    try:
        j = r.json()
    except Exception as e:
        print(f"[HTTP {r.status_code}] Non-JSON response for {intent}: {e}")
        print("Body preview:", r.text[:400])
        raise
    if not j.get("ok"):
        raise RuntimeError(f"Agent error [{intent}]: {j.get('error')}")
    return j

print("Ready. Using agent:", AGENT)

Ready. Using agent: http://127.0.0.1:8000/collect


In [12]:
# Cell 2 — leagues.list
resp = call_agent("leagues.list", {})
leagues = resp["data"].get("leagues") or []
print("Total leagues:", len(leagues))
print("\nFirst 10:")
for L in leagues[:34]:
    print(f"- {L.get('idLeague')}  {L.get('strLeague')}")

Total leagues: 34

First 10:
- 4328  English Premier League
- 4329  English League Championship
- 4330  Scottish Premier League
- 4331  German Bundesliga
- 4332  Italian Serie A
- 4334  French Ligue 1
- 4335  Spanish La Liga
- 4336  Greek Superleague Greece
- 4337  Dutch Eredivisie
- 4338  Belgian Pro League
- 4339  Turkish Super Lig
- 4340  Danish Superliga
- 4344  Portuguese Primeira Liga
- 4346  American Major League Soccer
- 4347  Swedish Allsvenskan
- 4350  Mexican Primera League
- 4351  Brazilian Serie A
- 4354  Ukrainian Premier League
- 4355  Russian Football Premier League
- 4356  Australian A-League
- 4358  Norwegian Eliteserien
- 4359  Chinese Super League
- 4367  _No League
- 4394  Italian Serie B
- 4395  Scottish Championship
- 4396  English League 1
- 4397  English League 2
- 4398  Italian Serie C Girone C
- 4399  German 2. Bundesliga
- 4400  Spanish La Liga 2
- 4401  French Ligue 2
- 4403  Swedish Superettan
- 4404  Brazilian Serie B
- 4406  Argentinian Primera Division


In [13]:
# Cell 3 — teams.list (by leagueName → name-based lookup under the hood)
LEAGUE_NAME = "French Ligue 2"  # change to test others

resp = call_agent("teams.list", {"leagueName": LEAGUE_NAME})
teams = resp["data"].get("teams") or []
print(f"Teams for '{LEAGUE_NAME}': {len(teams)}")
print([t.get("strTeam") for t in teams[:12]])

# show a couple of records to confirm fields look right (league & id)
for t in teams[2:3]:
    print("\n— sample —")
    print("idTeam   :", t.get("idTeam"))
    print("strTeam  :", t.get("strTeam"))
    print("strLeague:", t.get("strLeague"))
    print("idLeague :", t.get("idLeague"))

Teams for 'French Ligue 2': 18
['Amiens', 'Annecy', 'Bastia', 'Boulogne', 'Clermont Foot', 'Grenoble', 'Guingamp', 'Laval', 'Le Mans', 'Montpellier', 'Nancy Lorraine', 'Pau FC']

— sample —
idTeam   : 133933
strTeam  : Bastia
strLeague: French Ligue 2
idLeague : 4401


In [14]:
# Cell 4 — team.get via teamName (+ leagueName narrowing)
LEAGUE_NAME = "English Premier League"  # change as needed
TEAM_NAME   = "Arsenal"                 # change as needed

# First confirm the list contains the team (so you know name is valid)
tlist = call_agent("teams.list", {"leagueName": LEAGUE_NAME})["data"].get("teams") or []
assert any(t.get("strTeam") == TEAM_NAME for t in tlist), f"{TEAM_NAME} not found in {LEAGUE_NAME}"

# Now fetch team details by name (agent resolves name → id → lookupteam.php)
resp = call_agent("team.get", {"teamName": TEAM_NAME})
team = resp["data"].get("team") or {}
print("Team:", team.get("strTeam"), "| idTeam:", team.get("idTeam"))
print("League:", team.get("strLeague"), "| idLeague:", team.get("idLeague"))
print("Country:", team.get("strCountry"))
print("Stadium:", team.get("strStadium"))
print("Website:", team.get("strWebsite"))

# If you want to see all columns:
print("\nColumns:", sorted(team.keys()))

Team: Arsenal | idTeam: 133604
League: English Premier League | idLeague: 4328
Country: England
Stadium: Emirates Stadium
Website: www.arsenal.com

Columns: ['idAPIfootball', 'idESPN', 'idLeague', 'idLeague2', 'idLeague3', 'idLeague4', 'idLeague5', 'idLeague6', 'idLeague7', 'idTeam', 'idVenue', 'intFormedYear', 'intLoved', 'intStadiumCapacity', 'strBadge', 'strBanner', 'strColour1', 'strColour2', 'strColour3', 'strCountry', 'strDescriptionCN', 'strDescriptionDE', 'strDescriptionEN', 'strDescriptionES', 'strDescriptionFR', 'strDescriptionHU', 'strDescriptionIL', 'strDescriptionIT', 'strDescriptionJP', 'strDescriptionNL', 'strDescriptionNO', 'strDescriptionPL', 'strDescriptionPT', 'strDescriptionRU', 'strDescriptionSE', 'strDivision', 'strEquipment', 'strFacebook', 'strFanart1', 'strFanart2', 'strFanart3', 'strFanart4', 'strGender', 'strInstagram', 'strKeywords', 'strLeague', 'strLeague2', 'strLeague3', 'strLeague4', 'strLeague5', 'strLeague6', 'strLeague7', 'strLocation', 'strLocked', '

In [16]:
# Cell 5 — players.list by teamName (agent resolves name → id, then lookup_all_players.php)
TEAM_NAME = "Annecy"  # try "Bayern Munich", "Barcelona", etc.

resp = call_agent("players.list", {"teamName": TEAM_NAME})
players = resp["data"].get("players") or []
print(f"Players returned for '{TEAM_NAME}':", len(players))

# See if the payload is actually for the chosen club (this caught the Arsenal duplication issue)
teams_in_payload = sorted({p.get("strTeam") for p in players if p})
print("Team labels in payload:", teams_in_payload)

# Quick sample
for p in players[:10]:
    print("-", p.get("idPlayer"), "|", p.get("strPlayer"), "|", p.get("strTeam"), "|", p.get("strPosition"))

# Inspect idTeam vs idTeam2 distribution
idTeam_vals  = sorted({p.get("idTeam") for p in players if p.get("idTeam")})
idTeam2_vals = sorted({p.get("idTeam2") for p in players if p.get("idTeam2")})
print("\nUnique idTeam values   :", idTeam_vals[:10], ("… +more" if len(idTeam_vals) > 10 else ""))
print("Unique idTeam2 values  :", idTeam2_vals[:10], ("… +more" if len(idTeam2_vals) > 10 else ""))

# All columns union (handy to decide what you want to surface)
all_keys = set().union(*(p.keys() for p in players)) if players else set()
print("\nColumns (players):", sorted(all_keys))

Players returned for 'Annecy': 29
Team labels in payload: ['Arsenal']
- 34175641 | Albert Sambi Lokonga | Arsenal | Central Midfield
- 34163698 | Ben White | Arsenal | Right-Back
- 34169884 | Bukayo Saka | Arsenal | Right Winger
- 34164499 | Christian Nørgaard | Arsenal | Defensive Midfield
- 34194118 | Cristhian Mosquera | Arsenal | Centre-Back
- 34148681 | David Raya | Arsenal | Goalkeeper
- 34161584 | Declan Rice | Arsenal | Defensive Midfield
- 34200626 | Ethan Nwaneri | Arsenal | Attacking Midfield
- 34176976 | Fábio Vieira | Arsenal | Attacking Midfield
- 34160962 | Gabriel Jesus | Arsenal | Centre-Forward

Unique idTeam values   : ['133604'] 
Unique idTeam2 values  : ['0', '133901', '133906', '133907', '133909', '133910', '133913', '133914', '133915', '133916'] … +more

Columns (players): ['dateBorn', 'dateDied', 'dateSigned', 'idAPIfootball', 'idESPN', 'idPlayer', 'idPlayerManager', 'idTeam', 'idTeam2', 'idTeamNational', 'idTransferMkt', 'idWikidata', 'intLoved', 'intSoccerXMLT

In [24]:
LEAGUE_NAME = "Belgian Pro League"   # change as needed
SEASON      = "2014-2015"            # must exist for the chosen league

# seasons (sanity)
resp = call_agent("seasons.list", {"leagueName": LEAGUE_NAME})
seasons = resp.get("data", {}).get("seasons") or []
print("Seasons available:", len(seasons), "| sample:", [s.get("strSeason") for s in seasons[:5]])

# events for season
resp = call_agent("events.list", {"leagueName": LEAGUE_NAME, "season": SEASON})
events = resp.get("data", {}).get("events") or []
print(f"\nEvents for {LEAGUE_NAME} {SEASON}: {len(events)}")
for e in events[:10]:
    print("-", e.get("idEvent"), "|", e.get("dateEvent"), "|", e.get("strEvent"))

Seasons available: 14 | sample: ['2012-2013', '2013-2014', '2014-2015', '2015-2016', '2016-2017']

Events for Belgian Pro League 2014-2015: 100
- 444581 | 2014-07-25 | Standard vs Charleroi
- 444582 | 2014-07-26 | Waasland-Beveren vs Club Brugge
- 444583 | 2014-07-26 | Westerlo vs Lokeren
- 444584 | 2014-07-26 | Cercle Brugge vs Gent
- 444585 | 2014-07-26 | Lierse vs KV Oostende
- 444586 | 2014-07-27 | Mechelen vs Genk
- 444587 | 2014-07-27 | Anderlecht vs Mouscron-Peruwelz
- 444588 | 2014-07-27 | Waregem vs Kortrijk
- 444589 | 2014-08-01 | KV Oostende vs Anderlecht
- 444590 | 2014-08-02 | Kortrijk vs Standard


In [33]:
# Change this to any match name the season list returned (e.g., from eventsseason.php)
EVENT_NAME = "Standard vs Charleroi"  # <- change

resp = call_agent("event.get", {"eventName": EVENT_NAME})
data = resp.get("data", {}) if resp else {}
cands = data.get("candidates") or []

print(f"Candidates for '{EVENT_NAME}': {len(cands)}")
for c in cands[:20]:
    print("-", c.get("idEvent"), "|", c.get("dateEvent"), "|", c.get("strSeason"), "|", c.get("strEvent"))

# Keep one id for next cell (pick any from the printed list)
CHOSEN_ID = cands[0]["idEvent"] if cands else None
print("CHOSEN_ID =", CHOSEN_ID)

Candidates for 'Standard vs Charleroi': 14
- 2239080 | 2025-05-04 | 2024-2025 | Standard vs Charleroi
- 2076917 | 2024-10-20 | 2024-2025 | Standard vs Charleroi
- 1823946 | 2023-12-16 | 2023-2024 | Standard vs Charleroi
- 1578244 | 2023-04-14 | 2022-2023 | Standard vs Charleroi
- 1245541 | 2021-12-05 | 2021-2022 | Standard vs Charleroi
- 1015182 | 2021-01-24 | 2020-2021 | Standard vs Charleroi
- 603925 | 2019-09-29 | 2019-2020 | Standard vs Charleroi
- 577014 | 2018-09-15 | 2018-2019 | Standard vs Charleroi
- 524130 | 2017-09-10 | 2017-2018 | Standard vs Charleroi
- 510450 | 2016-08-19 | 2016-2017 | Standard vs Charleroi
- 488030 | 2016-02-20 | 2015-2016 | Standard vs Charleroi
- 461209 | 2012-12-07 | 2012-2013 | Standard vs Charleroi
- 457452 | 2013-10-20 | 2013-2014 | Standard vs Charleroi
- 444581 | 2014-07-25 | 2014-2015 | Standard vs Charleroi
CHOSEN_ID = 2239080


In [34]:
# Use the same EVENT_NAME as Cell 1, and set EVENT_ID to one of the ids shown above
EVENT_NAME = "Lierse vs KV Oostende"  # <- same as Cell 1
EVENT_ID   = 457456                          # or paste an id from Cell 1 output

args = {
    "eventName": EVENT_NAME,
    "eventId": str(EVENT_ID),          # <- this filters candidates by id
    "expand": ["timeline","stats","lineup"]
}
resp = call_agent("event.get", args)
data = resp.get("data", {}) if resp else {}

# Summary
ev  = data.get("event") or {}
tl  = data.get("timeline") or []
st  = data.get("stats") or []
lu  = data.get("lineup") or []

print("\n=== Event Detail ===")
for k in ["idEvent","dateEvent","strEvent","strLeague","strSeason",
          "strHomeTeam","strAwayTeam","intHomeScore","intAwayScore","strStatus","strVideo"]:
    print(f"{k:14} {ev.get(k)}")

print("\nTimeline rows:", len(tl))
print("Stats rows   :", len(st))
print("Lineup rows  :", len(lu))


=== Event Detail ===
idEvent        457456
dateEvent      2013-10-26
strEvent       Lierse vs KV Oostende
strLeague      Belgian Jupiler League
strSeason      2013-2014
strHomeTeam    Lierse
strAwayTeam    KV Oostende
intHomeScore   0
intAwayScore   2
strStatus      None
strVideo       None

Timeline rows: 12
Stats rows   : 12
Lineup rows  : 22
